---------------

# **Tweets, Users, Sentiment, and Key Phrases**

----------------

In [1]:
date_input = "2023-05-21" 

# **1. Imports**

In [3]:
import ads
import os
import sqlalchemy 
import oci
import tweepy
import time
import numpy as np
import pandas as pd
from collections import Counter
from pandas import json_normalize 
import cx_Oracle
from ads.database import connection 
from ads.database.connection import Connector
from sqlalchemy import create_engine
from urllib.request import urlopen
import configparser
import re
import random
import shutil
from zipfile import ZipFile
from tempfile import NamedTemporaryFile
import urllib
import requests
import json
from cleantext import clean
import datetime
from sqlalchemy import create_engine

print("Imports are done")

Imports are done


# **2. Authenticate Against OCI**

In [4]:
from oci.config import from_file
config = oci.config.from_file(file_location='~/.oci/config', profile_name='DEFAULT')

# **4. Get latest Tweets from Twitter**

In [6]:
## Riders and team only. Not staff
list_of_twitter = ["@soudalquickstep", "#soudalquickstep", "(from:soudalquickstep)",
#                   "@EthanVernon22", 
                     "@cernyjosef",
#                    "@alafpolak1",
#                     "@k_asgreen",
#                   "@AndreaBagioli",
                    "@ballero_94",
                   "@cattamat",
# # #                   "@remicav",
#                    "@Tim_Declercq",
# # #                   "@3sdevenyns",
                    "@EvenepoelRemco",
                     "@HirtJan",
#                     "@FabioJakobsen",                
# #                   "@JamesKnoxx",
# #                   "@yveslampaert",
#                    "@masnada_fausto",
# #                   "@MerlierTim",
#                   "@MichaelMorkov",
#                   "@pedersencasp",
#                   "@mauro_schmid",
#                   "@flosenech",
                   "@Pieter_Serry",
#                   "@SteimleJannik",
#                   "@sv_rtin",
#                   "@bert_bvl",
#                   "@StanVanTricht",
                    "@IlanWilder",
#                   "@MVansevenant99",
#                   "@EthanVernon22",
                  "@LouisVervaeke"
                  ]

In [ ]:
# Auth
bearer_token = "XXXXXXXXXXXXXXXXXXXX"

# API endpoint
search_url = "https://api.twitter.com/2/tweets/search/recent"

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

######
######
######
###### loop through list of twitter
######
######
######
######

########## blocks to loop through. 6 blocks of 4 hours.
#time_blocks = ["T00:00:01Z", "T04:00:01Z", "T08:00:01Z", "T12:00:01Z", "T16:00:01Z", "T20:00:01Z"]  #6 x 4 hours

time_blocks_30min = ["T00:00:01Z", "T00:30:01Z", "T01:00:01Z", "T01:30:01Z", "T02:00:01Z", "T02:30:01Z", "T03:00:01Z", "T03:30:01Z",   # 4 hours 1
                     "T04:00:01Z", "T04:30:01Z", "T05:00:01Z", "T05:30:01Z", "T06:00:01Z", "T06:30:01Z", "T07:00:01Z", "T07:30:01Z",   # 4 hours 2
                     "T08:00:01Z", "T08:30:01Z", "T09:00:01Z", "T09:30:01Z", "T10:00:01Z", "T10:30:01Z", "T11:00:01Z", "T11:30:01Z",   # 4 hours 3
                     "T12:00:01Z", "T12:30:01Z", "T13:00:01Z", "T13:30:01Z", "T14:00:01Z", "T14:30:01Z", "T15:00:01Z", "T15:30:01Z",   # 4 hours 4
                     "T16:00:01Z", "T16:30:01Z", "T17:00:01Z", "T17:30:01Z", "T18:00:01Z", "T18:30:01Z", "T19:00:01Z", "T19:30:01Z",   # 4 hours 5
                     "T20:00:01Z", "T20:30:01Z", "T21:00:01Z", "T21:30:01Z", "T22:00:01Z", "T22:30:01Z", "T23:00:01Z", "T23:30:01Z"    # 4 hours 6  
                    ]  #30 min blocks

#empty dataframe
appended_data  = pd.DataFrame()

for time in time_blocks_30min:
    
    #get the date (day/month) and time block as date
    date = date_input + time

    for account in list_of_twitter:
        try:  
            #query_params 
            query_params= {'query': account,
                           'start_time': date,
                            'place.fields':'contained_within,country,country_code,full_name,id,name,place_type',
                            'tweet.fields': 'author_id,created_at,context_annotations', 
                           'user.fields':'username,location', 
                           'expansions':'author_id,in_reply_to_user_id,entities.mentions.username,referenced_tweets.id.author_id', 
                           'max_results':'100'}

            #invoke response. Json response
            json_response = connect_to_endpoint(search_url, query_params)

            #see results. Push to a dataframe
            input_tweets = json_normalize(json_response, 'data')

            #clean tweets, add account columsn on the individual dataframe
            input_tweets.insert(0, 'time_block', date)
            input_tweets.insert(0, 'account', account)
            print(account + " has tweets: " + str(input_tweets.shape[0]) + " in time block " + date)

            #append dataframe to overall dataframe           
            appended_data = pd.concat([appended_data, input_tweets], axis=0).reset_index(drop=True)
        
        except:  #when error in no tweets (data = empty) skip
            pass
            
#clean text
appended_data['text'] = appended_data['text'].str.replace(',|\$|\.|\\|\…|\||\!|\-|\'|\?|\:|\#|\@|\n|\_|\(|\)', '')

#delete duplicate tweets
df_input_tweets = appended_data.drop_duplicates(subset=["id"], keep="first")

#add changed date/timestamp column
df_input_tweets['timeStamp'] = pd.to_datetime(df_input_tweets['created_at'], format='%Y/%m/%d %H:%M:%S')
df_input_tweets['timeStamp'] = df_input_tweets['timeStamp'].map(lambda x: str(x)[:-6])

print("Number of total tweets in (with duplicates)" + str(appended_data.shape[0]))
print("Number of total tweets in (without duplicates)" + str(df_input_tweets.shape[0]))

200
@soudalquickstep has tweets: 100 in time block 2023-05-21T00:00:01Z
200
#soudalquickstep has tweets: 20 in time block 2023-05-21T00:00:01Z
200
(from:soudalquickstep) has tweets: 57 in time block 2023-05-21T00:00:01Z
200
200
@ballero_94 has tweets: 100 in time block 2023-05-21T00:00:01Z
200
200
@EvenepoelRemco has tweets: 100 in time block 2023-05-21T00:00:01Z
200
200
@Pieter_Serry has tweets: 100 in time block 2023-05-21T00:00:01Z
200
@IlanWilder has tweets: 100 in time block 2023-05-21T00:00:01Z
200
200
@soudalquickstep has tweets: 100 in time block 2023-05-21T00:30:01Z
200
#soudalquickstep has tweets: 20 in time block 2023-05-21T00:30:01Z
200
(from:soudalquickstep) has tweets: 57 in time block 2023-05-21T00:30:01Z
200
200
@ballero_94 has tweets: 100 in time block 2023-05-21T00:30:01Z
200
200
@EvenepoelRemco has tweets: 100 in time block 2023-05-21T00:30:01Z
200
200
@Pieter_Serry has tweets: 100 in time block 2023-05-21T00:30:01Z
200
@IlanWilder has tweets: 100 in time block 2023-

# **5. Get user information**

### Get user information, like their user name
- Users who made/produced the Tweet
- If replied to a Tweet, who did the use reply to?

## **5.1 User who Tweeted**

In [ ]:
#copy tweets dataframe
dataset_tweets = df_input_tweets

#remove duplicates reply to to limit request
dataset_tweets.drop_duplicates(subset=['author_id'], keep='first', inplace=True)

# Auth
bearer_token = "AAAAAAAAAAAAAAAAAAAAAKNyJQEAAAAAqnvkEua0SASyd4UH%2FaY9oy09aDY%3D4UU34UOrNBUQyRzEPsKeYCkqv5FCAzIUOzycIVxnyY9FfHoGs4"

# API endpoint
search_url = "https://api.twitter.com/2/users/"

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url):
    response = requests.get(url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

####################

list_users = []

for x in dataset_tweets['author_id']:
    
    try:
    
        #get specific user information
        url = search_url + x + "?user.fields=public_metrics"
        json_response = connect_to_endpoint(url)

        #normalize response
        xs = list(json_response.values())
        #dataset_user = json_normalize(json_response)
        
        author_id = xs[0]["id"]
        user_name = xs[0]["username"]
        name = xs[0]["name"]
        followers_count = xs[0]["public_metrics"]["followers_count"]
        following_count = xs[0]["public_metrics"]["following_count"]
        tweet_count = xs[0]["public_metrics"]["tweet_count"]
        listed_count = xs[0]["public_metrics"]["listed_count"]

        #append response to list
        list_users.append([author_id, user_name, name, followers_count, following_count, tweet_count, listed_count])
        #list_users.append(dataset_user)
    
    except:  #when error in no tweets (data = empty) skip
        pass
    
    
#make new dataframe and append list
dataset_tweets_app = dataset_tweets

#list to df and append to existing df
list_users_reshaped = np.array(list_users).reshape(-1,7)
list_users_df = pd.DataFrame(list_users_reshaped, columns = ['author_id', 'username', 'name', 'followers_count', 'following_count', 'tweet_count', 'listed_count'])

#join dataframes
#dataset_tweets_and_user = pd.concat([dataset_tweets_app, list_users_df], axis=1, join="inner")


## **5.2 In reply to user information**

In [ ]:
#remove duplicates reply to to limit request
reply_to_inpt = df_input_tweets.drop_duplicates(subset=['in_reply_to_user_id'], keep='first')

list_users_in_reply_to = []

for x in reply_to_inpt['in_reply_to_user_id']:
    try:
        
        if pd.isna(x) == False:

            #get specific user information
            url = search_url + str(x)
            json_response = connect_to_endpoint(url)

            #normalize response
            dataset_user = json_normalize(json_response)

            #append response to list
            list_users_in_reply_to.append(dataset_user)
                #print(list_users_in_reply_to) 
    
        else:
            print("No reply to someone")

    except:
        pass
    
#list to df and append to existing df
list_users_reshaped_x = np.array(list_users_in_reply_to).reshape(-1,3)
list_users_in_reply_to_df = pd.DataFrame(list_users_reshaped_x, columns = ['in_reply_to_user_id', 'name_reply_to', 'username_reply_to'])

# **6. Change / Append columns**

In [ ]:
##drop columns
dataset_tweets_dropped = df_input_tweets.drop(['edit_history_tweet_ids', 'context_annotations', 'referenced_tweets'], axis=1)

#add binary column. In reply to or retweet
dataset_tweets_dropped['retweet_or_reply'] = ["reply" if pd.isna(x) == False else "retweet" for x in dataset_tweets_dropped['in_reply_to_user_id']]
dataset_tweets_dropped["tweet_count"] = 1

# **7. Sentiment Analysis - Using OCI AI Service - Language**

## **7.1 Sentiment extraction**

In [ ]:
#create dataset to use in sentiment
dataset = dataset_tweets_dropped

# Initialize service client with default config file
ai_language_client = oci.ai_language.AIServiceLanguageClient(config)

#empty list
results = []

# Iterate over Pandas DF and apply Sentiment Detection to each row
for index, row in dataset.iterrows():

    textx = row["text"]
    keyx = row["id"]

    # ai call
    batch_detect_language_sentiments_response = ai_language_client.batch_detect_language_sentiments(
        batch_detect_language_sentiments_details=oci.ai_language.models.BatchDetectLanguageSentimentsDetails(
            documents=[
                oci.ai_language.models.TextDocument(
                    key = keyx,
                    text = textx,
                    language_code="en")],
        ),    level=["SENTENCE"])

    # Extract Language Sentiments
    formatted_response = batch_detect_language_sentiments_response.data.documents

    rowid = formatted_response[0].key
    sentenceSentiment = formatted_response[0].document_sentiment
    positiveScore = formatted_response[0].document_scores["Positive"]
    neutralScore = formatted_response[0].document_scores["Neutral"]
    negativeScore = formatted_response[0].document_scores["Negative"]
    sentenceLength = formatted_response[0].sentences[0].length

    # Append the extracted attributes to the results list
    results.append([rowid, sentenceSentiment, positiveScore, neutralScore, negativeScore, sentenceLength])
    
    #print(results)

# Convert list of lists to a pandas df
resultsDF = pd.DataFrame(results, columns = ['id', 'Sentiment', 'PositiveScore', 'NeutralScore', 'NegativeScore', 'sentenceLength'])
resultsDF.head()

## **7.2 Append dataframe with sentiment per tweet**

In [ ]:
#merge dfs
df_out_1 = dataset_tweets_dropped.merge(resultsDF, how='left', on='id')

#remove duplicates tweets. ID is unique
df_out = df_out_1.drop_duplicates(subset=['id'], keep='first')

#drop entities.mentions
df_tweets_output = df_out.drop(['entities.mentions'], axis=1)

print("Shape should be " + str(df_input_tweets.shape))
print("Output shape is " + str(df_tweets_output.shape))

# **8. Extract entities from tweets and put in different table**

In [ ]:
results_list_entities = []

for index, row in df_input_tweets.iterrows():
    
    dummy = row['entities.mentions']
    
    try:
        if len(dummy)==1:
            username = dummy[0]['username']
            username2 = np.nan
            username3 = np.nan
            results_list_entities.append([row['id'], username, username2, username3])
        
        elif len(dummy)==2:
            username = dummy[0]['username']
            username2 = dummy[1]['username']
            username3 = np.nan
            results_list_entities.append([row['id'], username, username2, username3])           
            
        else:
            username = dummy[0]['username']
            username2 = dummy[1]['username']
            username3 = dummy[2]['username']

            results_list_entities.append([row['id'], username, username2, username3])
    except:
        pass
            
mentions_df = pd.DataFrame(results_list_entities, columns=['id', 'metion_1', 'mention_2', 'mention_3'])
mentions_df.tail(10)

# **9. Push results to database**

## **9.1 Connection to ADW**

In [ ]:
database_name = 'ADWGIRO'
database_user = 'XXXXX'
database_password = 'XXXX'
wallet_storage_directory = '/home/datascience/ADW/' 

# Create the wallet directory if missing: 
ads.set_documentation_mode(False)

os.makedirs(wallet_storage_directory, mode=0o700, exist_ok=True)

wallet_path = os.path.join(wallet_storage_directory, database_name)

# Prepare to store ADB connection information
adb_config = os.path.join(wallet_storage_directory, '.credentials')

# Write a configuration file for login creds.
config = configparser.ConfigParser()
config.read(adb_config)
config[database_name] = {'tns_admin': wallet_path,
                         'sid': 'adwgiro_high',
                         'user': database_user,
                         'password': database_password}
with open(adb_config, 'w') as configfile:
    config.write(configfile)

# Read in the credentials configuration files
my_config = configparser.ConfigParser()
my_config.read(adb_config)

# Limit the information to a specific database
my_creds = my_config[database_name]

# extract the wallet
wallet_file = 'Wallet_{}.zip'.format(database_name)
wallet_filename = os.path.join(wallet_storage_directory, wallet_file)
if not os.path.exists(wallet_filename):
    print("The file {} does not exist.".format(wallet_filename))
    print("Please copy the Wallet file, {}, into the directory {} then rerun this cell.".format(wallet_file, wallet_filename))
else:
    os.makedirs(wallet_path, mode=0o700, exist_ok=True)
    with ZipFile(wallet_filename, 'r') as zipObj:
        zipObj.extractall(wallet_path)
        
        
# Update the sqlnet.ora

sqlnet_path = os.path.join(wallet_path, 'sqlnet.ora')
sqlnet_original_path = os.path.join(wallet_path, 'sqlnet.ora.original')
sqlnet_backup_path = os.path.join(wallet_path, 'sqlnet.ora.backup')
if not os.path.exists(sqlnet_original_path):
    shutil.copy(sqlnet_path, sqlnet_original_path)
if os.path.exists(sqlnet_path):
    shutil.copy(sqlnet_path, sqlnet_backup_path)
sqlnet_re = re.compile('(WALLET_LOCATION\s*=.*METHOD_DATA\s*=.*DIRECTORY\s*=\s*\")(.*)(\".*)', 
                       re.IGNORECASE)
tmp = NamedTemporaryFile()
with open(sqlnet_path, 'rt') as sqlnet:
    for line in sqlnet:
        tmp.write(bytearray(sqlnet_re.subn(r'\1{}\3'.format(wallet_path), line)[0], 
                            encoding='utf-8'))
tmp.flush()
shutil.copy(tmp.name, sqlnet_path)
tmp.close()

# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Test the database connection
creds = config[database_name]
connect = 'sqlplus ' + creds.get('user') + '/' + creds.get('password') + '@' + creds.get('sid')
print(os.popen(connect).read())

#connection and engine
uri = 'oracle+cx_oracle://' + creds['user'] + ':' + creds['password'] + '@' + creds['sid']
os.environ['TNS_ADMIN'] = creds['tns_admin']
engine = create_engine(uri)

# **9.2 Reply Dataframe appending to database table**

### **9.3.1. Query previous table to remove duplicates before appending new, unqiue rows to the same table**

In [ ]:
#get entire table
df_reply_to = pd.read_sql('SELECT * from GIRO.soudalquickstep_reply_to', con=engine)

#join old and new tables
df_reply_to_concat = pd.concat([list_users_in_reply_to_df, df_reply_to])

# # #drop duplicates
df_reply_to_concat_to_database = df_reply_to_concat.drop_duplicates(subset=['in_reply_to_user_id'], keep='first')

#convert to string
df_reply_to_concat_to_database['in_reply_to_user_id'] = df_reply_to_concat_to_database['in_reply_to_user_id'].astype('str') 
df_reply_to_concat_to_database['name_reply_to'] = df_reply_to_concat_to_database['name_reply_to'].astype('str') 
df_reply_to_concat_to_database['username_reply_to'] = df_reply_to_concat_to_database['username_reply_to'].astype('str') 

### **9.3.2. Push to database new entries (each time replaces the same table name with unique values**

In [ ]:
#truncate table
Conn = engine.connect()
Conn.execute("TRUNCATE TABLE soudalquickstep_reply_to")

In [ ]:
print(df_reply_to_concat_to_database.shape)

#push results to database
df_reply_to_concat_to_database.to_sql('soudalquickstep_reply_to', con=engine, index=False, if_exists="append", dtype={
            'in_reply_to_user_id': sqlalchemy.types.NVARCHAR(length=255), 
            'name_reply_to': sqlalchemy.types.NVARCHAR(length=255), 
            'username_reply_to': sqlalchemy.types.NVARCHAR(length=255) })

# **9.3 Tweets_table**

In [ ]:
#change column names
df_tweets_output = df_tweets_output.rename(columns={'timeStamp': 'timestamp_x', 'Sentiment':'sentiment_x', 'PositiveScore':'positive_x', 'NeutralScore':'neutral_x', 'NegativeScore':'negative_x', 'sentenceLength':'sentence_length_x'})

In [ ]:
#get entire table
df_tweets_from_db = pd.read_sql('SELECT * from GIRO.soudalquickstep_tweets', con=engine)

#join old and new tables
df_tweets_concat = pd.concat([df_tweets_output, df_tweets_from_db])

# # #drop duplicates
df_tweets_concat_to_database = df_tweets_concat.drop_duplicates(subset=['id'], keep='first')

In [ ]:
#truncate table
Conn = engine.connect()
Conn.execute("TRUNCATE TABLE soudalquickstep_tweets")

In [ ]:
print(df_tweets_concat_to_database.shape)

df_tweets_concat_to_database.to_sql('soudalquickstep_tweets', con=engine, index=False, if_exists="append", dtype={
            'account': sqlalchemy.types.NVARCHAR(length=255), 
            'time_block': sqlalchemy.types.NVARCHAR(length=255), 
            'created_at': sqlalchemy.types.NVARCHAR(length=255), 
            'text': sqlalchemy.types.NVARCHAR(length=500), 
            'in_reply_to_user_id': sqlalchemy.types.NVARCHAR(length=255),
            'author_id': sqlalchemy.types.NVARCHAR(length=255), 
            'id': sqlalchemy.types.NVARCHAR(length=255), 
            'timestamp_x': sqlalchemy.types.NVARCHAR(length=255),  #sqlalchemy.DateTime()
#             'data_name': sqlalchemy.types.NVARCHAR(length=255), 
#             'data_username': sqlalchemy.types.NVARCHAR(length=255),
            'retweet_or_reply': sqlalchemy.types.NVARCHAR(length=255), 
            'tweet_count':  sqlalchemy.types.INTEGER(), 
            'sentiment_x': sqlalchemy.types.NVARCHAR(length=255), 
            'positive_x': sqlalchemy.types.Float(precision=3, asdecimal=True), 
            'neutral_x': sqlalchemy.types.Float(precision=3, asdecimal=True),
            'negative_x': sqlalchemy.types.Float(precision=3, asdecimal=True), 
            'sentence_length_x':  sqlalchemy.types.INTEGER()     })

# **9.4 Entities per tweet**

In [ ]:
#mentions_df.to_csv("./df_entities.csv")

In [ ]:
#get entire table
df_entities = pd.read_sql('SELECT * from GIRO.soudalquickstep_entities', con=engine)

#join old and new tables
df_entities_concat = pd.concat([mentions_df, df_entities])

# # #drop duplicates
df_entities_concat_to_datbase = df_entities_concat.drop_duplicates(subset=['id'], keep='first')

In [ ]:
#truncate table
Conn = engine.connect()
Conn.execute("TRUNCATE TABLE soudalquickstep_entities")

In [ ]:
print(df_entities_concat_to_datbase.shape)

#push results to database
df_entities_concat_to_datbase.to_sql('soudalquickstep_entities', con=engine, index=False, if_exists="append", dtype={
            'id': sqlalchemy.types.NVARCHAR(length=255), 
            'mention_1': sqlalchemy.types.NVARCHAR(length=255), 
            'mention_2': sqlalchemy.types.NVARCHAR(length=255),
            'mention_3': sqlalchemy.types.NVARCHAR(length=255)
})

# **9.5 Users data and nickname**

In [ ]:
#get entire table
df_users = pd.read_sql('SELECT * from GIRO.soudalquickstep_users', con=engine)

#change column names
list_users_df = list_users_df.rename(columns={'followers_count': 'user_followers_count', 'following_count': 'user_following_count', 'tweet_count': 'user_tweet_count', 'listed_count': 'user_listed_count'})

#join old and new tables
df_users_concat = pd.concat([list_users_df, df_users])

# # #drop duplicates
df_users_concat_to_datbase = df_users_concat.drop_duplicates(subset=['author_id'], keep='first')

In [ ]:
#truncate table
Conn = engine.connect()
Conn.execute("TRUNCATE TABLE soudalquickstep_users")

In [ ]:
print(df_users_concat_to_datbase.shape)

#push results to database
df_users_concat_to_datbase.to_sql('soudalquickstep_users', con=engine, index=False, if_exists="append", dtype={
            'author_id': sqlalchemy.types.NVARCHAR(length=255), 
            'username': sqlalchemy.types.NVARCHAR(length=255), 
            'name': sqlalchemy.types.NVARCHAR(length=255),
            'followers_count':  sqlalchemy.types.INTEGER(),
            'following_count':  sqlalchemy.types.INTEGER(),
            'tweet_count':  sqlalchemy.types.INTEGER(),
            'listed_count':  sqlalchemy.types.INTEGER()})

-----------------------------------

---

---